# **Required libraries**

In [ ]:
#installling required packages
!pip install --no-cache-dir -q torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install --no-cache-dir -q torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install --no-cache-dir -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install tensorflow-gpu

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
#importing required libraries
from scipy.stats import rankdata, iqr, trim_mean
from scipy.stats import rankdata
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error,mean_absolute_percentage_error
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score

import time
import math
import csv
import time
import logging
import random
from traceback import print_tb
import os
from datetime import datetime as dt
import more_itertools as mit
import argparse
import json
import sys
import itertools as it

import numpy as np
from numpy import percentile
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torch.nn as nn
from torch_geometric.nn import GCNConv, GATConv, EdgeConv
import torch.nn.functional as F
from torch.nn import Parameter, Linear, Sequential, BatchNorm1d, ReLU
import torch.nn.functional as F
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
from torch.utils.data import SequentialSampler
from torch_geometric.nn.inits import glorot, zeros

from prophet import Prophet
import plotly.graph_objects as go

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
from tensorflow.keras.callbacks import History, EarlyStopping
from joblib import Parallel, delayed

from keras.models import Sequential
from keras.callbacks import History, EarlyStopping, Callback
from keras.layers import LSTM, Dense, Activation, Dropout

logger = logging.getLogger('lstmndt')


# **LSTM-NDT**

In [ ]:

class Config:
    #to load parameters from config_dict into the global object

    def __init__(self, config_dict):
        self.dictionary = config_dict

        for k, v in self.dictionary.items():
            setattr(self, k, v)



def make_dirs():
    #to create directories for storing data if they don't already exist

    config = Config(config_lstmndt)

    paths = ['models', 'logs']

    for p in paths:
        if not os.path.isdir(p):
            os.mkdir(p)


def setup_logging():
    #Configuring the logging object to track training, and evaluation.

    logger = logging.getLogger('lstmndt')
    logger.setLevel(logging.INFO)

    stdout = logging.StreamHandler(sys.stdout)
    stdout.setLevel(logging.INFO)
    logger.addHandler(stdout)

    return logger





In [ ]:
class Channel:
    def __init__(self, config, chan_id):

        # To Load and reshape predicted values and actual and actual values for a channel

        self.id = chan_id
        self.config = config
        self.X_train = None #training inputs
        self.y_train = None #actual channel training values
        self.X_test = None #test inputs
        self.y_test = None #actual channel test values
        self.y_hat = None #forecast values
        self.train = None #train data loaded from .csv file
        self.test = None # test data loaded from .csv file

    def shape_data(self, arr, train=True):
        #Shape raw input streams for ingestion into LSTM. config.l_s specifies

        data = []
        for i in range(len(arr) - self.config.l_s - self.config.n_predictions):
            data.append(arr[i:i + self.config.l_s + self.config.n_predictions])
        data = np.array(data)

        assert len(data.shape) == 3

        if train:
            np.random.shuffle(data)
            self.X_train = data[:, :-self.config.n_predictions, :]
            self.y_train = data[:, -self.config.n_predictions:, 0]
        else:
            self.X_test = data[:, :-self.config.n_predictions, :]
            self.y_test = data[:, -self.config.n_predictions:, 0]


    def load_data(self, train_path=None, test_path=None):

      #Load train and test data from local CSV files.

      if train_path and test_path:
        train_data = pd.read_csv(train_path)
        test_data = pd.read_csv(test_path)

      elif train_path:
        data = pd.read_csv(train_path)
        train_data = data.sample(frac=0.8, random_state=42)
        test_data = data.drop(train_data.index)

      else:
        raise ValueError("Either train_path or both train_path and test_path must be provided.")

      # Extract the values of the channel for normalization
      train_values = train_data[self.id].values.reshape(-1, 1)
      test_values = test_data[self.id].values.reshape(-1, 1)

      # Perform robust scaling using RobustScaler
      scaler = RobustScaler()
      train_values_scaled = scaler.fit_transform(train_values)
      test_values_scaled = scaler.transform(test_values)

      # Assign the scaled values back to the train and test data
      train_data[self.id] = train_values_scaled
      test_data[self.id] = test_values_scaled

      self.train = train_data[self.id].values.reshape(-1, 1)
      self.test = test_data[self.id].values.reshape(-1, 1)

      self.shape_data(self.train)
      self.shape_data(self.test, train=False)


In [ ]:
class Errors:
    def __init__(self, channel, config, run_id):

        #Batch processing of errors between actual and predicted values for a channel.

        self.config = config
        self.window_size = self.config.window_size # number of trailing batches to use in error calculation
        self.n_windows = int((channel.y_test.shape[0] -
                              (self.config.batch_size * self.window_size))
                             / self.config.batch_size)

        self.i_anom = np.array([])  #indices of anomalies in channel test values
        self.E_seq = [] #array of (start, end) indices for each continuous anomaly sequence in test values
        self.anom_scores = [] #score indicating relative severity of each anomaly sequence in E_seq

        # raw prediction error
        self.e = [abs(y_h - y_t[0]) for y_h, y_t in
                  zip(channel.y_hat, channel.y_test)]  #errors in prediction (predicted - actual)

        smoothing_window = int(self.config.batch_size * self.config.window_size
                               * self.config.smoothing_perc)
        if not len(channel.y_hat) == len(channel.y_test):
            raise ValueError('len(y_hat) != len(y_test): {}, {}'
                             .format(len(channel.y_hat), len(channel.y_test)))

        # smoothed prediction error
        self.e_s = pd.DataFrame(self.e).ewm(span=smoothing_window) \
            .mean().values.flatten()

        # for values at beginning < sequence length, just use avg
        self.e_s[:self.config.l_s] = \
            [np.mean(self.e_s[:self.config.l_s * 2])] * self.config.l_s

        self.normalized = np.mean(self.e / np.ptp(channel.y_test))
        logger.info("normalized prediction error: {0:.2f}"
                    .format(self.normalized))

    def adjust_window_size(self, channel):

       # to decrease the historical error window size (h) if number of test values is limited

        while self.n_windows < 0:
            self.window_size -= 1
            self.n_windows = int((channel.y_test.shape[0]
                                 - (self.config.batch_size * self.window_size))
                                 / self.config.batch_size)
            if self.window_size == 1 and self.n_windows < 0:
                raise ValueError('Batch_size ({}) larger than y_test (len={}). '
                                 'Adjust in config.yaml.'
                                 .format(self.config.batch_size,
                                         channel.y_test.shape[0]))

    def merge_scores(self):
        '''
        If anomalous sequences from subsequent batches are adjacent they will
        automatically be combined. This combines the scores for these initial
        adjacent sequences (scores are calculated as each batch is processed)
        where applicable.
        '''

        merged_scores = []
        score_end_indices = []

        for i, score in enumerate(self.anom_scores):
            if not score['start_idx'] - 1 in score_end_indices:
                merged_scores.append(score['score'])
                score_end_indices.append(score['end_idx'])

    def process_batches(self, channel):

        #to loops through batches of values for a channel.


        self.adjust_window_size(channel)

        for i in range(0, self.n_windows + 1):
            prior_idx = i * self.config.batch_size
            idx = (self.config.window_size * self.config.batch_size) \
                  + (i * self.config.batch_size)
            if i == self.n_windows:
                idx = channel.y_test.shape[0]

            window = ErrorWindow(channel, self.config, prior_idx, idx, self, i)

            window.find_epsilon()
            window.find_epsilon(inverse=True)

            window.compare_to_epsilon(self)
            window.compare_to_epsilon(self, inverse=True)

            if len(window.i_anom) == 0 and len(window.i_anom_inv) == 0:
                continue

            window.prune_anoms()
            window.prune_anoms(inverse=True)

            if len(window.i_anom) == 0 and len(window.i_anom_inv) == 0:
                continue

            window.i_anom = np.sort(np.unique(
                np.append(window.i_anom, window.i_anom_inv))).astype('int')
            window.score_anomalies(prior_idx)

            # update indices to reflect true indices in full set of values
            self.i_anom = np.append(self.i_anom, window.i_anom + prior_idx)
            self.anom_scores = self.anom_scores + window.anom_scores

        if len(self.i_anom) > 0:
            # group anomalous indices into continuous sequences
            groups = [list(group) for group in
                      mit.consecutive_groups(self.i_anom)]
            self.E_seq = [(int(g[0]), int(g[-1])) for g in groups
                          if not g[0] == g[-1]]

            '''additional shift is applied to indices so that they represent the
            position in the original data array, obtained from the .csv files,
            and not the position on y_test'''
            self.E_seq = [(e_seq[0] + self.config.l_s,
                           e_seq[1] + self.config.l_s) for e_seq in self.E_seq]

            self.merge_scores()



In [ ]:
class ErrorWindow:
    '''
    Data and calculations for a specific window of prediction errors.
    Includes finding thresholds, pruning, and scoring anomalous sequences
    for errors and inverted errors (flipped around mean) - significant drops
    in values can also be anomalous.'''



    def __init__(self, channel, config, start_idx, end_idx, errors, window_num):
        self.i_anom = np.array([])
        self.E_seq = np.array([])
        self.non_anom_max = -1000000   #highest smoothed error value below epsilon
        self.i_anom_inv = np.array([])
        self.E_seq_inv = np.array([])
        self.non_anom_max_inv = -1000000  #highest smoothed error value below epsilon_inv

        self.config = config
        self.anom_scores = []

        self.window_num = window_num

        self.sd_lim = 12.0  #default number of standard deviations to use for threshold if no winner or too many anomalous ranges when scoring candidate thresholds
        self.sd_threshold = self.sd_lim  #number of standard deviations for calculation of best anomaly threshold
        self.sd_threshold_inv = self.sd_lim #for inverted channel values

        self.e_s = errors.e_s[start_idx:end_idx]
        self.mean_e_s = np.mean(self.e_s) #mean of smoothed errors
        self.sd_e_s = np.std(self.e_s) #sd of smoothed errors
        self.e_s_inv = np.array([self.mean_e_s + (self.mean_e_s - e)
                                 for e in self.e_s])

        self.epsilon = self.mean_e_s + self.sd_lim * self.sd_e_s  #threshold for e_s above which an error is considered anomalous
        self.epsilon_inv = self.mean_e_s + self.sd_lim * self.sd_e_s

        self.y_test = channel.y_test[start_idx:end_idx]  #actual values
        self.sd_values = np.std(self.y_test) #st dev of y_test

        self.perc_high, self.perc_low = np.percentile(self.y_test, [95, 5])  #the 95th percentile of y_test values, the 5th percentile of y_test values
        self.inter_range = self.perc_high - self.perc_low #the range between perc_high - perc_low

        # ignore initial error values until enough history for processing
        self.num_to_ignore = self.config.l_s * 2
        # if y_test is small, ignore fewer
        if len(channel.y_test) < 2500:
            self.num_to_ignore = self.config.l_s
        if len(channel.y_test) < 1800:
            self.num_to_ignore = 0

    def find_epsilon(self, inverse=False):
        '''
        Find the anomaly threshold that maximizes function representing tradeoff
        between the number of anomalies and anomalous ranges and the reduction
        in mean and st dev if anomalous points are removed from errors '''

        e_s = self.e_s if not inverse else self.e_s_inv

        max_score = -10000000

        for z in np.arange(2.5, self.sd_lim, 0.5):
            epsilon = self.mean_e_s + (self.sd_e_s * z)

            pruned_e_s = e_s[e_s < epsilon]

            i_anom = np.argwhere(e_s >= epsilon).reshape(-1,)
            buffer = np.arange(1, self.config.error_buffer)
            i_anom = np.sort(np.concatenate((i_anom,
                                            np.array([i+buffer for i in i_anom])
                                             .flatten(),
                                            np.array([i-buffer for i in i_anom])
                                             .flatten())))
            i_anom = i_anom[(i_anom < len(e_s)) & (i_anom >= 0)]
            i_anom = np.sort(np.unique(i_anom))

            if len(i_anom) > 0:
                # group anomalous indices into continuous sequences
                groups = [list(group) for group
                          in mit.consecutive_groups(i_anom)]
                E_seq = [(g[0], g[-1]) for g in groups if not g[0] == g[-1]]

                mean_perc_decrease = (self.mean_e_s - np.mean(pruned_e_s)) \
                                     / self.mean_e_s
                sd_perc_decrease = (self.sd_e_s - np.std(pruned_e_s)) \
                                   / self.sd_e_s
                score = (mean_perc_decrease + sd_perc_decrease) \
                        / (len(E_seq) ** 2 + len(i_anom))

                # sanity checks / guardrails
                if score >= max_score and len(E_seq) <= 5 and \
                        len(i_anom) < (len(e_s) * 0.5):
                    max_score = score
                    if not inverse:
                        self.sd_threshold = z
                        self.epsilon = self.mean_e_s + z * self.sd_e_s
                    else:
                        self.sd_threshold_inv = z
                        self.epsilon_inv = self.mean_e_s + z * self.sd_e_s

    def compare_to_epsilon(self, errors_all, inverse=False):
        '''
        Compare smoothed error values to epsilon (error threshold) and group
        consecutive errors together into sequences '''

        e_s = self.e_s if not inverse else self.e_s_inv
        epsilon = self.epsilon if not inverse else self.epsilon_inv

        # Check if scale of errors compared to values too small
        if not (self.sd_e_s > (.05 * self.sd_values) or max(self.e_s)
                > (.05 * self.inter_range)) or not max(self.e_s) > 0.05:
            return

        i_anom = np.argwhere((e_s >= epsilon) &
                             (e_s > 0.05 * self.inter_range)).reshape(-1,)

        if len(i_anom) == 0:
            return
        buffer = np.arange(1, self.config.error_buffer+1)
        i_anom = np.sort(np.concatenate((i_anom,
                                         np.array([i + buffer for i in i_anom])
                                         .flatten(),
                                         np.array([i - buffer for i in i_anom])
                                         .flatten())))
        i_anom = i_anom[(i_anom < len(e_s)) & (i_anom >= 0)]

        # if it is first window, ignore initial errors
        if self.window_num == 0:
            i_anom = i_anom[i_anom >= self.num_to_ignore]
        else:
            i_anom = i_anom[i_anom >= len(e_s) - self.config.batch_size]

        i_anom = np.sort(np.unique(i_anom))

        # capture max of non-anomalous values below the threshold
        batch_position = self.window_num * self.config.batch_size
        window_indices = np.arange(0, len(e_s)) + batch_position
        adj_i_anom = i_anom + batch_position
        window_indices = np.setdiff1d(window_indices,
                                      np.append(errors_all.i_anom, adj_i_anom))
        candidate_indices = np.unique(window_indices - batch_position)
        non_anom_max = np.max(np.take(e_s, candidate_indices))

        # group anomalous indices into continuous sequences
        groups = [list(group) for group in mit.consecutive_groups(i_anom)]
        E_seq = [(g[0], g[-1]) for g in groups if not g[0] == g[-1]]

        if inverse:
            self.i_anom_inv = i_anom
            self.E_seq_inv = E_seq
            self.non_anom_max_inv = non_anom_max
        else:
            self.i_anom = i_anom
            self.E_seq = E_seq
            self.non_anom_max = non_anom_max

    def prune_anoms(self, inverse=False):
        '''
        Remove anomalies that don't meet minimum separation from the next
        closest anomaly or error value'''

        E_seq = self.E_seq if not inverse else self.E_seq_inv
        e_s = self.e_s if not inverse else self.e_s_inv
        non_anom_max = self.non_anom_max if not inverse \
            else self.non_anom_max_inv

        if len(E_seq) == 0:
            return

        E_seq_max = np.array([max(e_s[e[0]:e[1]+1]) for e in E_seq])
        E_seq_max_sorted = np.sort(E_seq_max)[::-1]
        E_seq_max_sorted = np.append(E_seq_max_sorted, [non_anom_max])

        i_to_remove = np.array([])
        for i in range(0, len(E_seq_max_sorted)-1):
            if (E_seq_max_sorted[i] - E_seq_max_sorted[i+1]) \
                    / E_seq_max_sorted[i] < self.config.p:
                i_to_remove = np.append(i_to_remove, np.argwhere(
                    E_seq_max == E_seq_max_sorted[i]))
            else:
                i_to_remove = np.array([])
        i_to_remove[::-1].sort()
        i_to_remove = i_to_remove.astype(int)
        if len(i_to_remove) > 0:
            E_seq = np.delete(E_seq, i_to_remove, axis=0)

        if len(E_seq) == 0 and inverse:
            self.i_anom_inv = np.array([])
            return
        elif len(E_seq) == 0 and not inverse:
            self.i_anom = np.array([])
            return

        indices_to_keep = np.concatenate([range(e_seq[0], e_seq[-1]+1)
                                          for e_seq in E_seq])

        if not inverse:
            mask = np.isin(self.i_anom, indices_to_keep)
            self.i_anom = self.i_anom[mask]
        else:
            mask_inv = np.isin(self.i_anom_inv, indices_to_keep)
            self.i_anom_inv = self.i_anom_inv[mask_inv]

    def score_anomalies(self, prior_idx):
        '''
        Calculate anomaly scores based on max distance from epsilon
        for each anomalous sequence '''

        groups = [list(group) for group in mit.consecutive_groups(self.i_anom)]

        for e_seq in groups:

            score_dict = {
                "start_idx": e_seq[0] + prior_idx,
                "end_idx": e_seq[-1] + prior_idx,
                "score": 0
            }

            score = max([abs(self.e_s[i] - self.epsilon)
                         / (self.mean_e_s + self.sd_e_s) for i in
                         range(e_seq[0], e_seq[-1] + 1)])
            inv_score = max([abs(self.e_s_inv[i] - self.epsilon_inv)
                             / (self.mean_e_s + self.sd_e_s) for i in
                             range(e_seq[0], e_seq[-1] + 1)])

            # the max score indicates whether anomaly was from regular
            # or inverted errors
            score_dict['score'] = max([score, inv_score])
            self.anom_scores.append(score_dict)



In [ ]:

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


class Model:
    def __init__(self, config, run_id, channel):
        '''
        Loads/trains LSTM model and predicts future values for a given channel '''

        self.config = config
        self.chan_id = channel.id
        self.run_id = run_id
        self.y_hat = np.array([])
        self.model = None

        if not self.config.train:
            try:
                self.load(channel)
            except FileNotFoundError:
                path = os.path.join('models',self.chan_id + '.h5')
                logger.warning('Training new model, couldn\'t find existing '
                               'model at {}'.format(path))
                self.train_new(channel)
                self.save()
        else:
            self.train_new(channel)
            self.save()

    def load(self,channel):

        # to load model for channel.


        logger.info('Loading pre-trained model')
        self.model = load_model(os.path.join('models', self.chan_id + '.h5'))
        train_metrics = self.model.evaluate(channel.X_train, channel.y_train, verbose=0)
        self.mae = train_metrics[1]
        self.mse = train_metrics[2]
        self.mape = train_metrics[3]

    def train_new(self, channel):

        #Train LSTM model according to specifications in config

        cbs = [History(), EarlyStopping(monitor='val_loss',
                                        patience=self.config.patience,
                                        min_delta=self.config.min_delta,
                                        verbose=0)]

        with tf.device('/GPU:0'):
            self.model = Sequential()

            self.model.add(LSTM(
                self.config.layers[0],
                input_shape=(None, channel.X_train.shape[2]),
                return_sequences=True))
            self.model.add(Dropout(self.config.dropout))

            self.model.add(LSTM(
                self.config.layers[1],
                return_sequences=False))
            self.model.add(Dropout(self.config.dropout))

            self.model.add(Dense(
                self.config.n_predictions))
            self.model.add(Activation('linear'))

            self.model.compile(loss=self.config.loss_metric,
                               optimizer=self.config.optimizer, metrics=['mae','mse','mape'])


            history=self.model.fit(channel.X_train,
                           channel.y_train,
                           batch_size=self.config.lstm_batch_size,
                           epochs=self.config.epochs,
                           validation_split=self.config.validation_split,
                           callbacks=cbs,
                           verbose=True)
            self.mae = history.history['mae'][-1]
            self.mse = history.history['mse'][-1]
            self.mape = history.history['mape'][-1]

    def save(self):

        #Save trained model.
        self.model.save(os.path.join('models',
                                     '{}.h5'.format(self.chan_id)))

    def aggregate_predictions(self, y_hat_batch, method='first'):
        '''
         to aggregate predictions for each timestep. When predicting n steps
        ahead where n > 1, will end up with multiple predictions for a
        timestep'''

        agg_y_hat_batch = np.array([])

        for t in range(len(y_hat_batch)):

            start_idx = t - self.config.n_predictions
            start_idx = start_idx if start_idx >= 0 else 0

            # predictions pertaining to a specific timestep lie along diagonal
            y_hat_t = np.flipud(y_hat_batch[start_idx:t+1]).diagonal()

            if method == 'first':
                agg_y_hat_batch = np.append(agg_y_hat_batch, [y_hat_t[0]])
            elif method == 'mean':
                agg_y_hat_batch = np.append(agg_y_hat_batch, np.mean(y_hat_t))

        agg_y_hat_batch = agg_y_hat_batch.reshape(len(agg_y_hat_batch), 1)
        self.y_hat = np.append(self.y_hat, agg_y_hat_batch)

    def batch_predict(self, channel):

        # to trained LSTM model to predict test data arriving in batches

        num_batches = int((channel.y_test.shape[0] - self.config.l_s)
                          / self.config.batch_size)
        if num_batches < 0:
            raise ValueError('l_s ({}) too large for stream length {}.'
                             .format(self.config.l_s, channel.y_test.shape[0]))

        with tf.device('/GPU:0'):
            for i in range(0, num_batches + 1):
                prior_idx = i * self.config.batch_size
                idx = (i + 1) * self.config.batch_size

                if i + 1 == num_batches + 1:
                    # remaining values won't necessarily equal batch size
                    idx = channel.y_test.shape[0]

                X_test_batch = channel.X_test[prior_idx:idx]
                y_hat_batch = self.model.predict(X_test_batch)
                self.aggregate_predictions(y_hat_batch)

        self.y_hat = np.reshape(self.y_hat, (self.y_hat.size,))

        channel.y_hat = self.y_hat

        y_hat_df = pd.DataFrame({'Predicted': self.y_hat})

        return channel


In [ ]:

class LSTM_NDT:
  #for running anomaly detection over a group of channels
    #def __init__(self, config_dict, result_path='/content/', train_path=None, test_path=None):
    def __init__(self, config_dict, train_path=None, test_path=None):

        self.train_path = train_path
        self.test_path = test_path

        self.results = []
        self.result_df = None
        self.chan_df = None

        self.result_tracker = {
            'true_positives': 0,
            'false_positives': 0,
            'false_negatives': 0
        }

        self.config = Config(config_dict)
        self.y_hat = None

        '''if not self.config.predict and self.config.use_id:
            self.id = self.config.use_id
        else:
            self.id = dt.now().strftime('%Y-%m-%d_%H.%M.%S')'''
        self.id = dt.now().strftime('%Y-%m-%d_%H.%M.%S')

        # make_dirs(self.id)
        make_dirs()

        # Add logging FileHandler based on ID
        hdlr = logging.FileHandler('logs/%s.log' % self.id)
        formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
        hdlr.setFormatter(formatter)
        logger.addHandler(hdlr)

        #self.result_path = result_path

        data = pd.read_csv(train_path)
        chan_ids = data.columns[1:].tolist()

        self.chan_df = pd.DataFrame({"chan_id": chan_ids}, columns=["chan_id"])

        logger.info("{} channels found for processing.".format(len(self.chan_df)))



    def log_final_results(self):

      # Print final results at the end of the experiment.


      unique_indices = set()
      total_point_anomalies = 0
      for seq in self.result_df['anomaly_sequences']:
        for start, end in seq:
            for i in range(start, end + 1):
                if i not in unique_indices:
                    unique_indices.add(i)
                    total_point_anomalies += 1

      print('-----------------------------------')
      print('Final Results :')
      print('-----------------------------------')
      print('Best model : LSTM-NDT')
      print('Total channel sets evaluated: {}'.format(len(self.result_df)))
      print('Avg normalized prediction error: {}'.format(self.result_df['normalized_pred_error'].mean()))
      print('Total number of values evaluated: {}'.format(self.result_df['num_test_values'].sum()))
      print('Total collective anomalies found: {}'.format(self.result_df['n_predicted_anoms'].sum()))
      print('Total number of point anomalies: {}'.format(total_point_anomalies))
      non_empty_sequences = self.result_df['anomaly_sequences'].apply(lambda seq: seq if len(seq) > 0 else None)
      non_empty_sequences = non_empty_sequences.dropna()

      if not non_empty_sequences.empty:
        print('Anomaly sequences start/end indices:')
        for seq in non_empty_sequences:
            print(seq)
      else:
        print('No anomaly sequences found.')


    def test(self):

        #Initiate testing for all channels using saved models and GPU.

        gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
        config = tf.compat.v1.ConfigProto(gpu_options=gpu_options)
        self.config.train=False
        with tf.compat.v1.Session(config=config) as sess:
            for i, row in self.chan_df.iterrows():
                logger.info('Stream # {}: {}'.format(i + 1, row.chan_id))
                channel = Channel(self.config, row.chan_id)
                channel.load_data(self.train_path, self.test_path)

                if self.config.predict:
                    model = Model(self.config, self.id, channel)
                    channel = model.batch_predict(channel)

                errors = Errors(channel, self.config, self.id)
                errors.process_batches(channel)

                result_row = {
                    'run_id': self.id,
                    'chan_id': row.chan_id,
                    'num_test_values': len(channel.X_test),
                    'n_predicted_anoms': len(errors.E_seq),
                    'normalized_pred_error': errors.normalized,
                    'anom_scores': errors.anom_scores,
                    'mae': model.mae,
                    'mse': model.mse,
                    'mape': model.mape
                }

                result_row['anomaly_sequences'] = errors.E_seq
                self.results.append(result_row)

                self.result_df = pd.DataFrame(self.results)

            self.log_final_results()

    def train(self):

      #Train models for all channels using GPU
      gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
      config = tf.compat.v1.ConfigProto(gpu_options=gpu_options)

      models = []
      errors=[]
      with tf.compat.v1.Session(config=config) as sess:
        for i, row in self.chan_df.iterrows():
            logger.info('Stream # {}: {}'.format(i + 1, row.chan_id))
            channel = Channel(self.config, row.chan_id)
            channel.load_data(self.train_path, self.test_path)

            if self.config.predict:
                model = Model(self.config, self.id, channel)
                models.append(model)

            errors_row = {
                    'mae': model.mae,
                    'mse': model.mse,
                    'mape': model.mape
                }
            errors.append(errors_row)

        errors_df = pd.DataFrame(errors)
        mae = errors_df['mae'].mean()
        mse = errors_df['mse'].mean()
        mape = errors_df['mape'].mean()

        means_df = pd.DataFrame({'mae': [mae], 'mse': [mse], 'mape': [mape]})
        means_df.to_csv(os.path.join('models/', 'errors.csv'), index=False)

        return [mae,mse,mape]



# **GNN-GDN**

In [ ]:
class GraphLayer(MessagePassing):
    #Graph convolutional layer implementation
    def __init__(self, in_channels, out_channels,
                 negative_slope=0.2, dropout=0, bias=True, inter_dim=-1):
        super(GraphLayer, self).__init__(aggr='add')

        self.in_channels = in_channels #Number of input channels/features.
        self.out_channels = out_channels #Number of output channels/features.
        self.negative_slope = negative_slope #LeakyReLU angle of the negative slope. Default is 0.2.
        self.dropout = dropout
        self.lin = Linear(in_channels, out_channels, bias=False)
        self.a=Parameter(torch.Tensor(1, 4*out_channels))

        if bias:
            self.bias = Parameter(torch.Tensor( out_channels))
        else:
            self.register_parameter('bias',None)

        self.reset_parameters()

    def reset_parameters(self):
        #Reset layer parameters using Xavier/Glorot and zero initialization.
        glorot(self.lin.weight)

        glorot(self.a)
        zeros(self.bias)


    def forward(self, batch_mat, topk_edge, embedding):
        #Forward pass of the graph convolutional layer.

        x = self.lin(batch_mat)
        x = (x, x)

        out = self.propagate(topk_edge, x=x, embedding=embedding, topk_edge=topk_edge)
        if self.bias is not None:
            out = out + self.bias

        return out

    def message(self, x_i, x_j, edge_index_i, size_i, embedding, topk_edge):
        #Message propagation function for the graph convolutional layer.
        embedding_j=torch.empty(size=x_i.shape)
        embedding_i=torch.empty(size=x_i.shape)
        for i in range(len(topk_edge[0])):
            j_idx, i_idx=topk_edge[0][i], topk_edge[1][i]
            embedding_j[i]=embedding[j_idx]
            embedding_i[i]=embedding[i_idx]

        git = torch.cat((x_i, embedding_i), dim=-1)
        gjt = torch.cat((x_j, embedding_j), dim=-1)
        gijt=torch.cat((git,gjt),dim=-1)
        alpha= (self.a * gijt).sum(-1)
        alpha = alpha.view(-1, 1)
        alpha = F.leaky_relu(alpha, self.negative_slope)

        alpha = softmax(alpha, edge_index_i, num_nodes=size_i)
        alpha = F.dropout(alpha, p=self.dropout, training=self.training)
        return x_j * alpha

    def __repr__(self):
        # String representation of the GraphLayer class
        return '{}({}, {}, heads={})'.format(self.__class__.__name__,self.in_channels,self.out_channels, 1)

class OutLayer(nn.Module):

    #Output layer implementation.
    def __init__(self, in_num, node_num, layer_num, inter_num = 512):
        super(OutLayer, self).__init__()
        modules = []

        for i in range(layer_num):
            if i == layer_num-1:
                modules.append(nn.Linear( in_num if layer_num == 1 else inter_num, 1))
            else:
                layer_in_num = in_num if i == 0 else inter_num
                modules.append(nn.Linear( layer_in_num, inter_num ))
                modules.append(nn.BatchNorm1d(inter_num))
                modules.append(nn.ReLU())
        self.mlp = nn.ModuleList(modules)

    def forward(self, x):
        # Forward pass of the output layer.
        out = x
        for mod in self.mlp:
            if isinstance(mod, nn.BatchNorm1d):
                out = out.permute(0,2,1)
                out = mod(out)
                out = out.permute(0,2,1)
            else:
                out = mod(out)
        return out



class GNNLayer(nn.Module):
    #GNN layer implementation.
    def __init__(self, in_channel, out_channel, inter_dim):
        super(GNNLayer, self).__init__()
        self.graph = GraphLayer(in_channel, out_channel, inter_dim=inter_dim)
        self.bn = nn.BatchNorm1d(out_channel)
        self.relu = nn.ReLU()
        self.leaky_relu = nn.LeakyReLU()

    def forward(self, batch_mat,topk_edge, embedding):
        #Forward pass of the GNN layer
        out= self.graph(batch_mat, topk_edge, embedding)
        out = self.bn(out)
        return self.relu(out)

In [ ]:
class GDN(nn.Module):
    def __init__(self, full_edges, node_num, embed_dim=64, out_layer_inter_dim=256, input_dim=10, out_layer_num=1, topk=20):
        #Graph Deviation Network layer implementation.
        super(GDN,self).__init__()
        self.embedding = nn.Embedding(node_num,embed_dim)
        self.bn_outlayer_in = nn.BatchNorm1d(embed_dim)
        self.gnn_layer=GNNLayer(input_dim,embed_dim,2*embed_dim)
        self.topk = topk
        self.edges=full_edges
        self.out_layer = OutLayer(embed_dim, node_num, out_layer_num, inter_num = out_layer_inter_dim)
        self.dp = nn.Dropout(0.2)

    def init_params(self):
        nn.init.kaiming_uniform_(self.embedding.weight, a=math.sqrt(5))

    def forward(self, batch_tensor, org_edge_index):
        #Forward pass of the GDN model.

        batch_mat = batch_tensor.clone().detach()
        batch_size, node_num, feature_len = batch_tensor.shape
        batch_mat = batch_mat.view(-1, feature_len).contiguous()
        scalar_data_num=batch_mat.shape[0]
        edge_num = self.edges.shape[1]
        all_embeddings = self.embedding(torch.arange(node_num))
        weights = all_embeddings.detach().clone()
        cos_ji_mat = torch.matmul(weights, weights.T)
        normed_mat = torch.matmul(weights.norm(dim=-1).view(-1,1), weights.norm(dim=-1).view(1,-1))
        cos_ji_mat = cos_ji_mat / normed_mat
        topk_indices_ji = torch.topk(cos_ji_mat, self.topk, dim=-1)[1]

        gated_i = torch.arange(0, node_num).T.unsqueeze(1).repeat(1, self.topk).flatten().unsqueeze(0)
        gated_j = topk_indices_ji.flatten().unsqueeze(0)
        topk_edge = torch.cat((gated_j, gated_i), dim=0)

        gcn_out = self.gnn_layer(batch_mat, topk_edge, embedding=all_embeddings)
        x = gcn_out.view(batch_size, node_num, -1)

        indexes = torch.arange(0,node_num)
        out = torch.mul(x, self.embedding(indexes))

        out = out.permute(0,2,1)
        out = F.relu(self.bn_outlayer_in(out))
        out = out.permute(0,2,1)
        out = self.dp(out)
        out = self.out_layer(out)
        out = out.view(-1, node_num)

        return out

In [ ]:
def err_scores(test_result, val_result):
    #Compute error scores for test and validation results.
    np_test_result = np.array(test_result)
    np_val_result = np.array(val_result)

    all_scores =  None
    all_normals = None
    feature_num = np_test_result.shape[-1]

    labels = np_test_result[2, :, 0].tolist()

    for i in range(feature_num):
        test_re_list = np_test_result[:2,:,i]
        val_re_list = np_val_result[:2,:,i]

        scores = s_err_scores(test_re_list, val_re_list)
        normal_dist = s_err_scores(val_re_list, val_re_list)

        if all_scores is None:
            all_scores = scores
            all_normals = normal_dist
        else:
            all_scores = np.vstack((
                all_scores,
                scores
            ))
            all_normals = np.vstack((
                all_normals,
                normal_dist
            ))

    return all_scores, all_normals

def s_err_scores(test_res, val_res):
    #Compute Smoothed  error scores based on test and validation results
    test_predict, test_gt = test_res
    val_predict, val_gt = val_res

    n_err_mid, n_err_iqr = err_median_and_iqr(test_predict, test_gt)

    test_delta = np.abs(np.subtract(
                        np.array(test_predict).astype(np.float64),
                        np.array(test_gt).astype(np.float64)
                    ))
    epsilon=1e-2

    err_scores = (test_delta - n_err_mid) / ( np.abs(n_err_iqr) +epsilon)

    smoothed_err_scores = np.zeros(err_scores.shape)
    before_num = 3
    for i in range(before_num, len(err_scores)):
        smoothed_err_scores[i] = np.mean(err_scores[i-before_num:i+1])


    return smoothed_err_scores


def detect_anomalies(total_err_scores, test_result, topk=1):

    #Compute the best performance metrics based on error scores and test results and threshold
    total_features = total_err_scores.shape[0]

    topk_indices = np.argpartition(total_err_scores, range(total_features-topk-1, total_features), axis=0)[-topk:]

    total_topk_err_scores = []
    total_topk_err_scores = np.sum(np.take_along_axis(total_err_scores, topk_indices, axis=0), axis=0)

    thresholds = get_threshold(total_topk_err_scores, 400)
    pred_values = np.array(test_result[0])
    true_values = np.array(test_result[1])

    pred_values = pred_values.reshape(-1, pred_values.shape[-1])
    true_values = true_values.reshape(-1, true_values.shape[-1])

    pred_values = pred_values[:, :total_topk_err_scores.shape[0]]
    true_values = true_values[:, :total_topk_err_scores.shape[0]]

    mae = mean_absolute_error(true_values, pred_values)
    mse = mean_squared_error(true_values, pred_values)
    mape = np.mean(np.abs((true_values - pred_values) / true_values)) * 100.0

    th_i = np.argmax(total_topk_err_scores)
    threshold = thresholds[th_i]

    pred_labels = np.zeros(len(total_topk_err_scores))
    pred_labels[total_topk_err_scores > threshold] = 1
    num_anomalies = np.sum(pred_labels == 1)
    anomaly_indices = np.where(pred_labels == 1)[0]

    # Grouping the consecutive indices
    groups = []
    current_start = anomaly_indices[0]
    for i in range(1, len(anomaly_indices)):
      if anomaly_indices[i] != anomaly_indices[i-1] + 1:
        current_end = anomaly_indices[i-1]
        groups.append((current_start, current_end))
        current_start = anomaly_indices[i]

    current_end = anomaly_indices[-1]
    groups.append((current_start, current_end))
    print('-----------------------------------')
    print('Final Results :')
    print('-----------------------------------')
    print('Best model : GDN')
    print("Number of point anomalies:", num_anomalies)
    print("number of collective anomalies", len(groups))
    print("Grouped anomaly indices:")
    for start, end in groups:
      print([start, end])
    return mae, mse, mape, threshold

def get_threshold(scores, th_steps):
    #Compute the threshold values based on scores
    padding_list = [0]*(len(scores) - len(scores))
    if len(padding_list) > 0:
        scores = padding_list + scores

    scores_sorted = rankdata(scores, method='ordinal')
    th_steps = th_steps
    th_vals = np.array(range(th_steps)) * 1.0 / th_steps

    thresholds = [None] * th_steps
    for i in range(th_steps):
        score_index = scores_sorted.tolist().index(int(th_vals[i] * len(scores)+1))
        thresholds[i] = scores[score_index]

    return thresholds

def err_median_and_iqr(predicted, groundtruth):
    #Compute the median and IQR (Interquartile Range) of the errors.

    np_arr = np.abs(np.subtract(np.array(predicted), np.array(groundtruth)))
    err_median = np.median(np_arr)
    err_iqr = iqr(np_arr)

    return err_median, err_iqr

In [ ]:
#Loss functions
def loss_mse(y_pred, y_true):
    loss = F.mse_loss(y_pred, y_true, reduction='mean')

    return loss

def loss_mae(y_pred, y_true):
    loss = F.l1_loss(y_pred, y_true, reduction='mean')
    return loss

def loss_mape(y_pred, y_true):
    epsilon = 1e-7
    loss = torch.abs((y_true - y_pred) / (y_true + epsilon))
    loss = torch.mean(loss) * 100.0
    return loss



def train(model, config, train_dataloader, val_dataloader, nodes_list, test_dataloader, test_dataset, train_dataset, full_edges):

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=config['decay'])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1) # Example

    now = time.time()

    train_loss_list = []
    val_loss_list = []

    acu_loss = 0
    acu_mse = 0
    acu_mape = 0
    val_acu_loss = 0
    min_loss = 1e+8
    mse = 0
    mae = 0
    mape = 0

    i = 0
    epoch = config['epoch']
    early_stop_win = 15

    model.train()

    log_interval = 1000
    stop_improve_count = 0

    dataloader = train_dataloader

    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)

    for i_epoch in range(epoch):

        acu_loss = 0
        val_acu_loss = 0
        model.train()
        scheduler.step()

        for x, labels, _ in dataloader:
            _start = time.time()

            x, labels = [item.float() for item in [x, labels]]

            optimizer.zero_grad()
            out = model(x, full_edges).float()
            loss = loss_mae(out, labels)
            mse=loss_mse(out,labels)
            mape=loss_mape(out,labels)

            loss.backward()
            optimizer.step()


            acu_loss += loss.item()
            acu_mse += mse.item()
            acu_mape += mape.item()

            i += 1

        # append average training loss for the epoch to train_loss_list
        train_loss_list.append(acu_loss/len(dataloader))

        logger.info('epoch ({}/{}), Loss: {:.8f}, ACU_loss: {:.8f}'.format(
            i_epoch, epoch, acu_loss/len(dataloader), acu_loss))

        # use val dataset to judge
        if val_dataloader is not None:

            val_loss, val_result = test(model, val_dataloader,full_edges)

            val_acu_loss += val_loss

            if val_loss < min_loss:
                torch.save(model.state_dict(),'model.pt')

                min_loss = val_loss
                stop_improve_count = 0
                mae= acu_loss/len(dataloader)
                mse= acu_mse/len(dataloader)
                mape=acu_mape/len(dataloader)
            else:
                stop_improve_count += 1


            if stop_improve_count >= early_stop_win:
                break

        # append average validation loss for the epoch to val_loss_list
        if val_dataloader is not None:
            val_loss_list.append(val_acu_loss/len(val_dataloader))

    return train_loss_list, val_loss_list, mae,mse.item(),mape.item()


def test(model, dataloader, full_edges):
    # test
    loss_func = nn.MSELoss(reduction='mean')

    test_loss_list = []
    now = time.time()

    test_predicted_list = []
    test_ground_list = []
    test_labels_list = []

    t_test_predicted_list = []
    t_test_ground_list = []
    t_test_labels_list = []

    test_len = len(dataloader)

    model.eval()

    i = 0
    acu_loss = 0
    for x, y, labels in dataloader:
        x, y, labels= [item.float() for item in [x, y, labels]]

        with torch.no_grad():
            predicted = model(x, full_edges).float()
            loss = loss_mae(predicted, y)
            labels = labels.unsqueeze(1).repeat(1, predicted.shape[1])

            if len(t_test_predicted_list) <= 0:
                t_test_predicted_list = predicted
                t_test_ground_list = y
                t_test_labels_list = labels
            else:
                t_test_predicted_list = torch.cat((t_test_predicted_list, predicted), dim=0)
                t_test_ground_list = torch.cat((t_test_ground_list, y), dim=0)
                t_test_labels_list = torch.cat((t_test_labels_list, labels), dim=0)

        test_loss_list.append(loss.item())
        acu_loss += loss.item()

        i += 1

        if i % 100 == 0:
            logging.info('Processed {} batches.'.format(i))

    test_predicted_list = t_test_predicted_list.tolist()
    test_ground_list = t_test_ground_list.tolist()
    test_labels_list = t_test_labels_list.tolist()
    avg_loss = sum(test_loss_list)/len(test_loss_list)
    logging.info('Test completed. Average loss: {}'.format(avg_loss))

    return avg_loss, [test_predicted_list, test_ground_list, test_labels_list]


def log_final_results(test_result, val_result, report):
    feature_num = len(test_result[0][0])
    np_test_result = np.array(test_result)
    np_val_result = np.array(val_result)
    test_scores, normal_scores = err_scores(test_result, val_result)
    info = detect_anomalies(test_scores, np_test_result, topk=1)




In [ ]:

class TimeDataset(Dataset):

    def __init__(self, data_df, mode, config):
        self.data_df=data_df
        self.data_df = self.data_df.astype('float64')
        self.config=config
        self.mode=mode

        self.feature, self.label, self.attack = self.process()

    def process(self):
        win=self.config['slide_win']
        stride=self.config['slide_stride']

        if self.mode=='test':

            attack_col=torch.tensor(np.array(self.data_df['attack']), dtype=torch.float64)
            self.data_df=self.data_df.drop(columns=['attack'])

        num_nodes=len(self.data_df.columns)
        timestamp_len=len(self.data_df.iloc[:,2])

        ran=range(win,timestamp_len,stride) if self.mode =='train' else range(win,timestamp_len)
        data_num=len(ran)
        feature=torch.zeros((data_num,num_nodes,win))
        label=torch.zeros((data_num,num_nodes))
        attack=torch.zeros((data_num))

        for cnt,i in enumerate(ran):
            mat_i=torch.zeros((num_nodes,win))
            label_i=torch.zeros((num_nodes))
            for j in range(num_nodes):
                column = torch.tensor(np.array(self.data_df.iloc[:,j]), dtype=torch.float64)

                mat_i[j]=column[i-win:i]
                label_i[j]=column[i]

                if j==0 and self.mode=='test':
                    attack[cnt]=attack_col[i]

            feature[cnt]=mat_i
            label[cnt]=label_i

        return feature, label, attack

    def __len__(self):
        return len(self.feature)

    def __getitem__(self,idx):

        return self.feature[idx], self.label[idx], self.attack[idx]

def train_val_loader(train_dataset, batch, val_ratio=0.1, num_workers=0, seed=42):
    dataset_len = len(train_dataset)
    train_use_len = int(dataset_len * (1 - val_ratio))
    val_use_len = int(dataset_len * val_ratio)

    random.seed(seed)
    val_start_index = random.randrange(train_use_len)
    indices = torch.arange(dataset_len)

    train_sub_indices = torch.cat([indices[:val_start_index], indices[val_start_index+val_use_len:]])
    train_subset = Subset(train_dataset, train_sub_indices)

    val_sub_indices = indices[val_start_index:val_start_index+val_use_len]
    val_subset = Subset(train_dataset, val_sub_indices)

    train_dataloader = DataLoader(train_subset, batch_size=batch, shuffle=True, num_workers=num_workers)
    val_dataloader = DataLoader(val_subset, batch_size=batch, shuffle=False, num_workers=num_workers)

    return train_dataloader, val_dataloader

def get_full_edges(csv_path):
    with open(csv_path, 'r') as f:
        reader = csv.reader(f)
        nodes_list = next(reader)[1:]

    full_edges = []
    for i in range(len(nodes_list)):
        for j in range(len(nodes_list)):
            if i == j:
                continue
            full_edges.append([i, j])

    full_edges = torch.tensor(full_edges).T
    return full_edges,nodes_list

def load_data(config,train_path, test_path=None):
    if test_path is None:
        # Only train_path is given, read the CSV file and split into train and test
        df = pd.read_csv(train_path, index_col=0)
        df = df.rename(columns={df.columns[0]: "timestamp"})
        df["timestamp"] = range(len(df))
        train_df, test_df = train_test_split(df, test_size=0.2, shuffle=False)
    else:
        # Both train_path and test_path are given, read the respective CSV files
        train_df = pd.read_csv(train_path, index_col=0)
        train_df = train_df.rename(columns={train_df.columns[0]: "timestamp"})
        train_df["timestamp"] = range(len(train_df))

        test_df = pd.read_csv(test_path, index_col=0)
        test_df = test_df.rename(columns={test_df.columns[0]: "timestamp"})
        test_df["timestamp"] = range(len(test_df))

    # normalization
    scaler = RobustScaler()
    numerical_columns = train_df.select_dtypes(include=[np.number]).columns.tolist()
    train_df[numerical_columns] = scaler.fit_transform(train_df[numerical_columns])
    test_df[numerical_columns] = scaler.transform(test_df[numerical_columns])

    train_dataset = TimeDataset(train_df, 'train', config)
    test_dataset = TimeDataset(test_df, 'train', config)

    train_dataloader, val_dataloader = train_val_loader(train_dataset, config['batch'], config['val_ratio'])
    test_dataloader = DataLoader(test_dataset, batch_size=config['batch'], shuffle=False, num_workers=8)

    return train_dataset,test_dataset,train_dataloader, val_dataloader, test_dataloader

# **Prophet**

In [ ]:
class PROPHET:
    def __init__(self, dataset):
        self.dataset = dataset
        self.df_entire = None
        self.prophet_model = None
        self.entire_dataframe = None
        self.columns=None
        self.target_column=None

    def col_to_ds(self,target_column):
      #takes csv file and renames first date column into 'ds' and target column as 'y'
        file_path = f'/content/{self.dataset}.csv'
        self.df_entire = pd.read_csv(file_path)
        first_column_name = self.df_entire.columns[0]

        self.df_entire = self.df_entire.rename(columns={first_column_name: 'ds', target_column: 'y'})
        self.df_entire['ds'] = pd.to_datetime(self.df_entire['ds'])

    def train(self,columns):
      #Adds regression based on input columns
        self.prophet_model = Prophet()
        for i in range(len(columns)):
            print("**********************Column name for regression: ",columns[i])
            self.prophet_model.add_regressor(columns[i], standardize=False)
        self.prophet_model.fit(self.df_entire)

    def channel_predict(self, column):
      #creates univariate models for each regression column for future forecasting
        df_column = self.df_entire[['ds', column]].rename(columns={column: 'y'})

        m_column = Prophet()
        m_column.fit(df_column)

        future_column = m_column.make_future_dataframe(periods=365)
        forecasted_column = m_column.predict(future_column)

        fc = forecasted_column.drop(axis=1, labels=[
            'trend', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper', 'additive_terms',
            'additive_terms_lower', 'additive_terms_upper', 'weekly', 'weekly_lower', 'weekly_upper', 'multiplicative_terms', 'multiplicative_terms_lower',
            'multiplicative_terms_upper'
        ])

        fc.rename(columns={'yhat': column}, inplace=True)

        return fc

    def merge_forecasts(self, forecasts):
      #combines forecsted values with previous dataset based on date
      df_forecast = forecasts[0]
      for i in range(1, len(forecasts)):
         df_forecast = pd.merge(df_forecast, forecasts[i], on='ds')
         # Drop duplicate columns
         df_forecast = df_forecast.loc[:, ~df_forecast.columns.duplicated()]
      return df_forecast

    def aggregate_predictions(self, df_forecast):
      #predicted for future 365 points for univariate and combined with dataframe
        df_forecast_last_one_year = df_forecast[-365:]
        print(self.df_entire.index.is_unique)
        print(df_forecast_last_one_year.index.is_unique)
        self.df_entire.reset_index(drop=True, inplace=True)
        df_forecast_last_one_year.reset_index(drop=True, inplace=True)
        self.entire_dataframe = pd.concat([self.df_entire, df_forecast_last_one_year], ignore_index=True)


    def predict_target(self):
      #predicts multivariate target column values
        forecasting_for_oneyear = self.prophet_model.predict(self.entire_dataframe)
        return forecasting_for_oneyear

    def evaluate_performance(self, forecasted_data):
      #evaluates MSE,MAE,MAPE scores
        performance = pd.merge(self.df_entire, forecasted_data[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on='ds')


        performance_MAE = mean_absolute_error(performance['y'], performance['yhat'])

        performance_MAPE = mean_absolute_percentage_error(performance['y'], performance['yhat'])

        performance_MSE = mean_squared_error(performance['y'], performance['yhat'])

        return performance_MAE,performance_MSE, performance_MAPE

    def log_final_results(self,forecasted_data):
      #Prints all anomalies and their indices
        performance = pd.merge(self.df_entire, forecasted_data[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on='ds')

        performance['anomaly'] = performance.apply(lambda rows: 1 if ((rows.y < rows.yhat_lower) or (rows.y > rows.yhat_upper)) else 0, axis=1)
        print('-----------------------------------')
        print('Final Results :')
        print('-----------------------------------')
        print('Best model : PROPHET')
        anomalies = performance[performance['anomaly']==1].sort_values(by='ds')
        print("Total number of point anomalies: ",anomalies['ds'].count())
        pd.set_option('display.max_columns', None)

        anomaly_indices = anomalies.index.tolist()
        continuous_anomalies = []
        start_index = None

        for i in range(len(anomaly_indices)):
          if i == 0 or anomaly_indices[i] != anomaly_indices[i-1] + 1:
            if start_index is not None:
              if start_index == prev_index:
                continuous_anomalies.append(start_index)
              else:
                continuous_anomalies.append([start_index, prev_index])
            start_index = anomaly_indices[i]
          prev_index = anomaly_indices[i]

        if start_index is not None:
          if start_index == prev_index:
            continuous_anomalies.append(start_index)
          else:
            continuous_anomalies.append([start_index, prev_index])
        print("Total number of collective anomalies : ", len(continuous_anomalies))
        print("Indices of anomalies: ")
        for i in range(0, len(continuous_anomalies), 10):
          line = continuous_anomalies[i:i+10]
          print(line)

#**MAIN()**

In [ ]:
import threading
import warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
if __name__ == '__main__':


    # Disable all warnings
    warnings.filterwarnings("ignore")

    # Set TensorFlow to use GPU
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Replace '0' with the GPU index you want to use

    # Enable GPU memory growth to allocate memory on-demand
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)
    #os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    dataset = 'stock_data'
    train_path = "stock_data.csv"
    test_path = None

    target_column=input("Enter the target column: ")
    columns = input("Enter the columns (comma-separated) you want to add regression for: ").split(",")

    config_lstmndt = {
        "train": False,
        "predict": True,
        "batch_size": 70,
        "window_size": 30,
        "smoothing_perc": 0.05,
        "error_buffer": 100,
        "loss_metric": "mae",
        "optimizer": "adam",
        "validation_split": 0.2,
        "dropout": 0.3,
        "lstm_batch_size": 64,
        "epochs": 35,
        "layers": [80, 80],
        "patience": 10,
        "min_delta": 0.0003,
        "l_s": 250,
        "n_predictions": 10,
        "p": 0.13
    }

    config_gdn = {
        'slide_win': 15,
        'slide_stride': 5,
        'batch': 128,
        'dim': 64,
        'val_ratio': 0.1,
        'topk': 2,  # including itself.
        'out_layer_num': 1,
        'out_layer_inter_dim': 256,
        'decay': 0.001,
        'epoch': 100,
        'report': 'best'  # or 'val'
    }

    full_edges, nodes_list = get_full_edges(train_path)
    train_dataset,test_dataset,train_dataloader, val_dataloader, test_dataloader = load_data(config=config_gdn,train_path=train_path, test_path=test_path)


    prophet_model = PROPHET(dataset)
    prophet_model.col_to_ds(target_column)


    # Get the columns from the user


    lstmndt_model = LSTM_NDT(config_dict=config_lstmndt, train_path=train_path, test_path=test_path)
    gdn_model = GDN(full_edges, len(nodes_list),
                    embed_dim=config_gdn['dim'],
                    input_dim=config_gdn['slide_win'],
                    out_layer_num=config_gdn['out_layer_num'],
                    out_layer_inter_dim=config_gdn['out_layer_inter_dim'],
                    topk=config_gdn['topk'])

    def run_lstmndt():
        global mae_lstmndt, mse_lstmndt, mape_lstmndt
        if config_lstmndt['train']== True :
          mae_lstmndt, mse_lstmndt, mape_lstmndt = lstmndt_model.train()

        else:
          errors_df = pd.read_csv('models/errors.csv')
          mae_lstmndt = errors_df['mae'].values[0]
          mse_lstmndt = errors_df['mse'].values[0]
          mape_lstmndt = errors_df['mape'].values[0]
        print("Training LSTM-NDT : complete")

    def run_gdn():
        global train_log, val_log, mae_gdn, mse_gdn, mape_gdn

        logging.basicConfig(filename='training.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logging.info('Loading data and configuring model')
        train_log, val_log, mae_gdn, mse_gdn, mape_gdn = train(gdn_model, config_gdn, train_dataloader, val_dataloader,
                                                              nodes_list, test_dataloader, test_dataset,
                                                              train_dataset, full_edges)
        print("Training GDN : complete")


    def run_prophet():
        global forecasting_for_oneyear,mae_p,mse_p,mape_p
        prophet_model.train(columns)
        forecasts = []
        for column in columns:
          fc = prophet_model.channel_predict(column)
          forecasts.append(fc)
        df_forecast = prophet_model.merge_forecasts(forecasts)
        prophet_model.aggregate_predictions(df_forecast)
        forecasting_for_oneyear = prophet_model.predict_target()
        mae_p,mse_p,mape_p=prophet_model.evaluate_performance(forecasting_for_oneyear)
        print("Training LSTM-NDT : complete")



    start_time = time.time()
    t1 = threading.Thread(target=run_lstmndt)
    t2 = threading.Thread(target=run_gdn)
    t3 = threading.Thread(target=run_prophet)

    # Start the threads
    t1.start()
    t2.start()
    t3.start()
    # Wait for the threads to finish
    t1.join()
    t2.join()
    t3.join()

    '''if mae_gdn < mae_p and mae_gdn < mae_lstmndt :
        # Testing for GDN model
        _, test_result = test(gdn_model, test_dataloader, full_edges)
        _, val_result = test(gdn_model, val_dataloader, full_edges)
        log_final_results(test_result, val_result, config_gdn['report'])

        print ("mae : ", mae_gdn)
        print ("mse : ", mse_gdn)
        print ("mape : ", mape_gdn)

    elif mae_lstmndt < mae_gdn and mae_lstmndt < mae_p  :
        # Testing for LSTMNDT model
        lstmndt_model.test()
        print ("mae : ", mae_lstmndt)
        print ("mse : ", mse_lstmndt)
        print ("mape : ", mape_lstmndt)

    elif mae_p < mae_gdn and mae_p < mae_lstmndt :
        # testing for Prophet model
        prophet_model.log_final_results(forecasting_for_oneyear)
        print ("mae : ", mae_p)
        print ("mse : ", mse_p)
        print ("mape : ", mape_p)'''

    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution Time:", execution_time, "seconds")


1 Physical GPUs, 1 Logical GPUs
Enter the target column: Close
Enter the columns (comma-separated) you want to add regression for: High,Low,Open,Volume,Adj Close


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmzd92h2_/80lg0gnb.json


Training LSTM-NDT : complete
**********************Column name for regression:  High
**********************Column name for regression:  Low
**********************Column name for regression:  Open
**********************Column name for regression:  Volume
**********************Column name for regression:  Adj Close


DEBUG:cmdstanpy:input tempfile: /tmp/tmpmzd92h2_/ygybytld.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=18288', 'data', 'file=/tmp/tmpmzd92h2_/80lg0gnb.json', 'init=/tmp/tmpmzd92h2_/ygybytld.json', 'output', 'file=/tmp/tmpmzd92h2_/prophet_model5pq15m7b/prophet_model-20230617121529.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:15:29 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
2023-06-17 12:15:29,869 - __main__ - INFO - epoch (0/100), Loss: 0.66847930, ACU_loss: 4.67935508
2023-06-17 12:15:29,869 - __main__ - INFO - epoch (0/100), Loss: 0.66847930, ACU_loss: 4.67935508
INFO:__main__:epoch (0/100), Loss: 0.66847930, ACU_loss: 4.67935508
2023-06-17 12:15:29,961 - __main__ - INFO - epoch (1/100), Loss: 0.67060336, ACU_loss: 4.69422352
2023-06-17 12:15:29,961 - __main__ - INF

Training GDN : complete


12:16:11 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmzd92h2_/38k1p710.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmzd92h2_/jp8sh8fu.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=91853', 'data', 'file=/tmp/tmpmzd92h2_/38k1p710.json', 'init=/tmp/tmpmzd92h2_/jp8sh8fu.json', 'output', 'file=/tmp/tmpmzd92h2_/prophet_model5xyd27bc/prophet_model-20230617121612.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:16:12 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:16:15 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run

True
True
Training LSTM-NDT : complete
Execution Time: 70.46269512176514 seconds


In [ ]:
_, test_result = test(gdn_model, test_dataloader, full_edges)
_, val_result = test(gdn_model, val_dataloader, full_edges)
log_final_results(test_result, val_result, config_gdn['report'])

print ("mae : ", mae_gdn)
print ("mse : ", mse_gdn)
print ("mape : ", mape_gdn)

-----------------------------------
Final Results :
-----------------------------------
Best model : GDN
Number of point anomalies: 158
number of collective anomalies 8
Grouped anomaly indices:
[3, 9]
[16, 18]
[40, 63]
[66, 68]
[83, 96]
[111, 150]
[162, 163]
[171, 235]
mae :  0.6684792978422982
mse :  0.810350775718689
mape :  104.41462707519531


In [ ]:
lstmndt_model.test()
print ("mae : ", mae_lstmndt)
print ("mse : ", mse_lstmndt)
print ("mape : ", mape_lstmndt)

-----------------------------------
Final Results :
-----------------------------------
Best model : LSTM-NDT
Total channel sets evaluated: 6
Avg normalized prediction error: 0.21392233647334583
Total number of values evaluated: 5580
Total collective anomalies found: 1
Total number of point anomalies: 218
Anomaly sequences start/end indices:
[(400, 617)]
mae :  0.5493184924125671
mse :  0.5416778922080994
mape :  1311.21826171875


In [ ]:
prophet_model.log_final_results(forecasting_for_oneyear)
print ("mae : ", mae_p)
print ("mse : ", mse_p)
print ("mape : ", mape_p)

-----------------------------------
Final Results :
-----------------------------------
Best model : PROPHET
Total number of point anomalies:  746
Total number of collective anomalies :  566
Indices of anomalies: 
[20, 168, 210, 231, 11, 13, 19, 21, 104, 146]
[211, 232, 31, 36, 22, 83, 170, 212, 233, 55]
[61, 1, 23, 148, [70, 71], [74, 75], [77, 78], 82, 149, 191]
[[96, 97], [99, 100], 102, 113, [119, 120], [123, 124], 65, 193, 235, 136]
[139, 141, [144, 145], 194, 236, 162, [166, 167], 26, 67, 151]
[173, 237, 177, 183, [186, 187], 88, 198, [200, 201], 203, 6]
[[221, 222], 225, 228, 230, 7, 48, 90, 154, 219, 239]
[[241, 243], [245, 246], [248, 249], 335, 464, [264, 267], [269, 272], 274, 465, 282]
[[285, 291], 316, 379, 401, 443, 302, [306, 312], 254, 276, 317]
[402, 444, 324, 329, 331, [333, 334], 255, 380, 423, 445]
[348, [350, 351], [354, 355], 296, 381, 424, 446, 467, 488, [367, 370]]
[[373, 375], 378, 257, 277, 297, 361, 382, 403, 425, 489]
[[386, 387], 389, 395, 398, 278, 298, 40